# Driving a Financial Valuation from a Design Scenario

```{warning}
The following is a work in progress. More content will be added as produced.
```

As seen in
[Loading a Design Scenario](../load_design_scenario.ipynb),
a design scenario's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to use the object model (graph) to
drive a financial valuation of the design scenario. This is done by calculating
cash flows per `Entity`, and then using the `Relationship`s to aggregate those
`Flow`s into appropriate `Stream`s.

## Load the Design Scenario
We will use the same design scenario as in the previous notebook.

In [1]:
import os

import locale
import pandas as pd
import networkx as nx
from IPython.display import IFrame
import plotly.subplots
import plotly.offline as py

import rangekeeper as rk

In [2]:
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN'))
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)


 SpeckleClient( server: https://speckle.xyz, authenticated: True )


### Inspect the Model Graph

In [3]:
model = speckle.get_commit(stream_id=stream_id)
parsed = rk.api.Speckle.parse(base=model['@scenario'])
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='design_scenario',
    type='scenario')

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


In [4]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html


## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design scenario.

In this example, revenue-based cash flows are generated by `Entity`s that have a
measureable floor area -- in this case, anything that has a 'gfa' property.

We can query the graph to find them:

### Revenue-producing Entities
Revenue-producing `Entity`s will be aggregated by their spatial containment. So
we can first slice our graph to a tree of nodes that have "contains"
relationships:

In [5]:
spatial_containment = scenario.filter_by_type(
    relationship_type='spatiallyContains',
    name='spatial_containment')

Note: we anticipate that the spatial decomposition is completely hierarchical
(i.e., no spaces overlap, which means no spaces have multiple parents). We can
check this by testing whether the containment graph is a "tree" (or
"arborescence": see [NetworkX Tree](https://networkx.org/documentation/stable/reference/algorithms/tree.html#tree)).

In [6]:
nx.is_arborescence(spatial_containment.graph)

True

In [7]:
# Plot the spatial containment graph
spatial_containment.plot(name='assets/spatial_containment')
IFrame(src="./spatial_containment.html", width='100%', height=800)

assets/spatial_containment.html


#### Simple Aggregation
Now that we have verified this, we can continue with a simple aggregation. We
wish to aggregate up through our hierarchy all `Entity`s that have a Gross Floor
Area ('gfa') property. *Rangekeeper* provides defaults for the `Entity.aggregate()`
method for this, whereby a named property of any `Entity` is summed (numerical
addition) into its parent `Entity` against a specified label:

In [8]:
scenario.aggregate(
    property='gfa',
    label='subtotal_gfa',
    relationship_type='spatiallyContains')

We can tabulate this by converting the graph to a pandas `DataFrame`. Note that
the graph nodes (as witnessed in the DataFrame's index and 'parent' column)
are GUID strings. Each of these have corresponding `Entity` objects, organised
via NetworkX's Node Attributes. See [NetworkX Tutorial](https://networkx.org/documentation/stable/tutorial.html#what-to-use-as-nodes-and-edges)

In [9]:
df = spatial_containment.to_DataFrame()
df = df[['name', 'type', 'gfa', 'subtotal_gfa', 'parent', 'use', 'ffl', 'number']]
df

,name,type,gfa,subtotal_gfa,parent,use,ffl,number
a39ac4c3-094b-4d92-b71d-46c5a857f407,development,development,NaN,45816.419959,None,NaN,NaN,NaN
981c0743-6a4c-4abd-8a97-2c937e48b897,buildingB,building,NaN,13436.884829,a39ac4c3-094b-4d92-b71d-46c5a857f407,NaN,NaN,NaN
e3149444-dfbc-41d0-9eb1-f08785a62da1,plinth,building,NaN,12773.744211,a39ac4c3-094b-4d92-b71d-46c5a857f407,NaN,NaN,NaN
529ce7bc-addf-4bad-8ec5-cbecbd73d4f1,buildingA,building,NaN,19605.790919,a39ac4c3-094b-4d92-b71d-46c5a857f407,NaN,NaN,NaN
29b746f5-42fe-4321-9a4a-8c65c9f0410c,utilities,utilities,NaN,0.000000,a39ac4c3-094b-4d92-b71d-46c5a857f407,NaN,NaN,NaN
3fc29315-81e7-45a9-8241-62938e904d73,buildingBresidential,space,NaN,11442.736624,981c0743-6a4c-4abd-8a97-2c937e48b897,residential,NaN,NaN
f508623d-1d81-4082-85ee-5b1e32c7bb10,buildingBparking,space,NaN,1178.164752,981c0743-6a4c-4abd-8a97-2c937e48b897,parking,NaN,NaN
6ac8ab6d-a730-44b8-97aa-a5db09b7fbd0,buildingBretail,space,NaN,815.983454,981c0743-6a4c-4abd-8a97-2c937e48b897,retail,NaN,NaN
71fd4551-8e65-4e7a-8e2a-9d18e69146b0,buildingBcores,utilities,NaN,0.000000,981c0743-6a4c-4abd-8a97-2c937e48b897,cores,NaN,NaN
c11f68da-0544-487b-9cb9-8a3a3a98a9b8,buildingBresidentialFloor0,floor,765.941631,765.941631,3fc29315-81e7-45a9-8241-62938e904d73,NaN,0.3,0.0


We can also plot this as a hierarchical pie chart (a.k.a. 'sunburst' chart), or
a treemap:

In [10]:
sunburst = spatial_containment.sunburst(property='subtotal_gfa')
treemap = spatial_containment.treemap(property='subtotal_gfa')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]])
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='spatial_containment_chart.html'

py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800 * len(fig.data))

#### Aggregating a (Financial) Calculation
A more complex aggregation involves collecting specific cash `Flow`s into
`Stream`s per `Entity`/`Assembly`; for instance, we may want to aggregate all
revenues generated by lettable floorspace into their respective parent `Entity`s,
in order to analyse or compare the performance of the scenario at different
resolutions.

We start by identifying which `Entity`s are 'floor' types and have an area
measurement:

In [11]:
floors = scenario.filter_by_type(entity_type='floor')
floors = [floor for floor in floors.get_entities().values() if hasattr(floor, 'gfa')]

For each floor we can project a set of simple cash flows, in a similar manner
to [A Basic DCF Valuation](../basic_dcf.ipynb).

First initialize our locale, and a set of parameters, including some around
area efficiency ratios, initial incomes per area, and growth rates:

In [12]:
locale.setlocale(locale.LC_ALL, 'en_au')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
period_type = rk.periodicity.Type.YEAR

In [13]:
efficiency_ratios = dict(
    office=0.88,
    retail=0.75,
    residential=0.82,
    parking=0.93
    )
initial_income_per_area_pa = dict(
    office=750,
    retail=1000,
    residential=600,
    parking=0
    )
pgi_growth_rates = dict(
    office=.02,
    retail=.05,
    residential=.035,
    parking=0
    )
vacancy_rates = dict(
    office=.075,
    retail=.5,
    residential=.02,
    parking=0,
    )

In [14]:
revenue_inputs = pd.DataFrame(
    data=dict(
        efficiency_ratio=efficiency_ratios,
        initial_income_per_area_pa=initial_income_per_area_pa,
        pgi_growth_rate=pgi_growth_rates,
        vacancy_rate=vacancy_rates,
        ))
revenue_inputs

,efficiency_ratio,initial_income_per_area_pa,pgi_growth_rate,vacancy_rate
office,0.88,750,0.020,0.075
retail,0.75,1000,0.050,0.500
residential,0.82,600,0.035,0.020
parking,0.93,0,0.000,0.000


In [15]:
params = dict(
    start_date=pd.Timestamp('2001-01-01'),
    num_periods=10,
    period_type=rk.periodicity.Type.YEAR
    )

##### `Stream`s of Revenue `Flow`s per `Entity`:
It is now simple to construct a set of cash flows per `Entity` in our design
scenario. We will use a similar structure to the previous notebooks.

First we will define a general 'Spans' class that defines the time periods. This
can be used by any subsequent calculations:

In [16]:
class Spans:
    def __init__(self, params: dict):
        self.params = params
        self.calc_span = rk.span.Span.from_num_periods(
            name='Span to Calculate Reversion',
            date=self.params['start_date'],
            period_type=self.params['period_type'],
            num_periods=self.params['num_periods'] + 1)
        self.acq_span = rk.span.Span.from_num_periods(
            name='Acquisition Span',
            date=rk.periodicity.offset_date(
                self.params['start_date'],
                num_periods=-1,
                period_type=self.params['period_type']),
            period_type=self.params['period_type'],
            num_periods=1)
        self.span = self.calc_span.shift(
            name='Span',
            num_periods=-1,
            period_type=self.params['period_type'],
            bound='end')

Then, we will define a `Revenues` class that holds the parameters:

In [17]:
class Revenues:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

And then we add Revenue `Flow`s:

In [18]:
@rk.update_class(Revenues)
class Revenues:
    def generate(self):
        self.pgi = rk.flux.Flow.from_projection(
            name='Potential Gross Income',
            value=self.params['initial_income'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['pgi_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.vacancy = rk.flux.Flow(
            name='Vacancy Allowance',
            movements=self.pgi.movements * -self.params['vacancy_rate'],
            units=currency.units)
        self.egi = rk.flux.Stream(
            name='Effective Gross Income',
            flows=[self.pgi, self.vacancy],
            period_type=self.params['period_type'])

Now we need to add `Entity`-specific parameters to the parameters dictionary,
and run  the `init_spans()` and `init_flows()` methods per floor `Entity`:

(Note we place the resultant `Stream` in the `events` attribute of the `Entity`)

In [19]:
spans = Spans(params)

In [20]:
for floor in floors:
    parent = floor.get_relatives(
        relationship_type='spatiallyContains',
        outgoing=False,
        assembly=spatial_containment)[0]
    floor['use'] = parent['use']

    floor.params = params.copy()
    floor.params['initial_income'] = initial_income_per_area_pa[floor['use']] * floor['gfa']
    floor.params['pgi_growth_rate'] = pgi_growth_rates[floor['use']]
    floor.params['vacancy_rate'] = vacancy_rates[floor['use']]

    # floor.events = {}

    revenues = Revenues(floor.params, spans)
    revenues.generate()

    floor['pgi'] = revenues.pgi
    floor['vacancy'] = revenues.vacancy
    floor['egi'] = revenues.egi
    # floor = {
    #     'pgi': revenues.pgi,
    #     'vacancy': revenues.vacancy,
    #     'egi': revenues.egi,
    #     }

##### Aggregating by Parent Container
Since we have the "containment" hierarchy, we can aggregate the `Revenues` per
parent `Entity`:

*Rangekeeper* provides some helper methods to assist in the aggregation of
`Stream`s and `Flow`s; specifically, since the `Entity.aggregate()` method
updates a specified `Entity`'s property with a dictionary of subentity
properties, keyed by their respective `entityId`s, we can use the
`aggregate_events()` function to support the collation of child `Entity`
`Flow`s into a single `Stream` per label in the `Entity`'s `events` property:

In [21]:
def aggregate_egi(**kwargs):
    return rk.graph.Entity.aggregate_events(
        **kwargs,
        property='egi',
        period_type=period_type)

This function is passed into the `Entity.aggregate()` method as the `function`
parameter:

In [22]:
spatial_containment.aggregate(
    function=aggregate_egis,
    property='events',
    label='subtotal_egi',
    relationship_type='spatiallyContains')

Now we can look at what the `aggregate` method has produced for a single floor:

In [23]:
floors[1]['events']['egi']

date,Potential Gross Income,Vacancy Allowance
2001,"$1,416,633.62","-$28,332.67"
2002,"$1,466,215.80","-$29,324.32"
2003,"$1,517,533.35","-$30,350.67"
2004,"$1,570,647.02","-$31,412.94"
2005,"$1,625,619.66","-$32,512.39"
2006,"$1,682,516.35","-$33,650.33"
2007,"$1,741,404.42","-$34,828.09"
2008,"$1,802,353.58","-$36,047.07"
2009,"$1,865,435.95","-$37,308.72"
2010,"$1,930,726.21","-$38,614.52"


We can then inspect the `aggregate_egi` property of an `Entity`, which holds a
`Stream` of `Flow`s (each coming from a child of that `Entity`). Let's use the
office compartment in Building A:

In [24]:
buildingAoffice = [e for (id, e) in spatial_containment.get_entities().items() if e.name == 'buildingAoffice'][0]
print(buildingAoffice.name)
buildingAoffice['subtotal_egi']

buildingAoffice


date,Effective Gross Income for a1fc1a28-1133-47f2-a8d9-58da817946c2 [buildingAofficeFloor1],Effective Gross Income for 0296748a-317a-47eb-bf94-e5dbbd78afc1 [buildingAofficeFloor2],Effective Gross Income for 74ed0525-d8f9-4f36-b20f-1b8a2baf60f2 [buildingAofficeFloor3],Effective Gross Income for ae0fb830-76df-4d91-82ed-4ed755fdec4b [buildingAofficeFloor4],Effective Gross Income for c1b696b1-cae0-49ed-a8ba-0e795438d3d2 [buildingAofficeFloor5],Effective Gross Income for add95b48-71c3-40e7-9f35-7db604f3f9f0 [buildingAofficeFloor6],Effective Gross Income for d8443e6d-9080-4154-89e3-29f0e98ed90a [buildingAofficeFloor7],Effective Gross Income for f8945b4f-6f59-4c5a-a83f-462a97f2dcc4 [buildingAofficeFloor8],Effective Gross Income for b548b184-1643-4c23-9b14-9025aeda59c3 [buildingAofficeFloor9],Effective Gross Income for e8048367-edd2-4a61-ae0f-fd878688c910 [buildingAofficeFloor10]
2001,"$1,145,271.71","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64"
2002,"$1,168,177.14","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44"
2003,"$1,191,540.69","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30"
2004,"$1,215,371.50","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29"
2005,"$1,239,678.93","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60"
2006,"$1,264,472.51","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59"
2007,"$1,289,761.96","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80"
2008,"$1,315,557.20","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94"
2009,"$1,341,868.34","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88"
2010,"$1,368,705.71","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67"


We can also easily sum and collapse that aggregation, as it is a Stream`:

In [25]:
buildingAoffice['subtotal_egi'].sum()

date,egi for bfc1c272-665f-432d-a7b3-a4c955556e7f [buildingAoffice] Aggregation (sum)
2001-12-31 00:00:00,"$10,191,978.50"
2002-12-31 00:00:00,"$10,395,818.07"
2003-12-31 00:00:00,"$10,603,734.43"
2004-12-31 00:00:00,"$10,815,809.12"
2005-12-31 00:00:00,"$11,032,125.30"
2006-12-31 00:00:00,"$11,252,767.81"
2007-12-31 00:00:00,"$11,477,823.16"
2008-12-31 00:00:00,"$11,707,379.63"
2009-12-31 00:00:00,"$11,941,527.22"
2010-12-31 00:00:00,"$12,180,357.76"


In [26]:
buildingAoffice['subtotal_egi'].sum().collapse()

date,egi for bfc1c272-665f-432d-a7b3-a4c955556e7f [buildingAoffice] Aggregation (sum)
2011-12-31 00:00:00,"$124,023,285.90"


The above subtotals the Effective Gross Income (EGI) for the office space of
Building A over the period of the scenario.

We can then do this for all `Entity`s in the graph, and plot the results:

In [27]:
sunburst = spatial_containment.sunburst(property='subtotal_egi')
treemap = spatial_containment.treemap(property='subtotal_egi')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]])
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='aggregate_egis_chart.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800 * len(fig.data))

### Cost-generating Entities
Let's now do the same for the cost-generating `Entity`s. While it is routine to
register revenue-generating `Flow`s with `Entity`s that have lettable or
sellable floor space, the convention to also register all cost-generating
`Flow`s with only those same `Entity`s is possibly only done out of convenience
rather than accuracy.

For example, both operating and capital expenses may be attributed to building
systems (e.g. maintenance, upgrading, & replacement of MEP or facade
componentry) that are not coincident with the floor space they are servicing.
Ie, the convention to average (sub)total costs across (sub)total floor space may
be misleading, especially when attempting to compare different scenarios at
different resolutions (See {cite}`farevuu2018`, 5.4 Flaw of Averages).

By virtue of *Rangekeeper*'s multi-faceted approach to its object model, costs
can be attributed and aggregated to `Entity`s that are not spatially coincident
with the revenue-generating `Entity`s; in the example design scenario, we will
use the 'services' relationship-type to register and aggregate some of these,
while space-specific costs will be registered and aggregated via spatial
containment:

#### Operational and Capital Expenses of Building Utilities & Systems
Of course, in a simplified model and without knowing design and engineering
details for the scenario, costs for operational and capital expenses will be
calculated from some proportional rate of spatial characteristics of the
building. In this example, we will use the following as simplifications of how
these types of costs may be calculated. Note that this is more a demonstration
of methodology, and in practice would adjust to real-world data about the
specific building systems and their costs.
-  'Floorplate-derived' Operational and Capital Expenses will be calculated as a
function of the subtotal floorplate area of 'building' `Entity`s. One could
think of this as representing the cost of operating the floor's MEP, or
maintaining, repairing and upgrading elements on the floors themselves
(e.g. FF&E)
- 'Facade-derived' Operational Expenses will be calculated as a function of the
  surface area of 'building' `Entity`. This would represent the cost of facade
maintenance, repair or replacement.
- 'Utility-derived' Capital Expenses will be calculated as a function of the
- volume of 'utility' `Entity`s. This represents the cost of repairing and
upgrading the building's MEP and transportation systems.


#### Cost `Flow`s per `Entity`
There are some properties of `Entity`s that we need to produce before we can
calculate `Flow`s off them. In this case we need to produce the facade area
for any `Entity`s that have 'perimeter' and 'ftf' (Floor-to-Floor height)
properties:


In [28]:
# Aggregate Facade Areas:
for (entityId, entity) in scenario.get_entities().items():
    if hasattr(entity, 'perimeter') & hasattr(entity, 'ftf'):
        entity['facade_area'] = entity['perimeter'] * entity['ftf']
spatial_containment.aggregate(
    property='facade_area',
    label='subtotal_facade_area')

We now set up operational and capital costing parameters that are based off
either areas or volume, as well as growth rates:

In [29]:
floor_opex_per_area_pa = dict(
    office=-200,
    retail=-300,
    residential=-150,
    parking=-75,
    )
facade_opex_per_area_pa = dict(
    office=-50,
    retail=-100,
    residential=-50,
    parking=-25,
    )
opex_growth_rate = 0.03
floor_capex_per_area_pa = dict(
    office=-125,
    retail=-175,
    residential=-125,
    parking=-50,
    )
utility_capex_per_vol_pa = -200
capex_growth_rate = 0.03

Now we can set up our `Costs` class to produce cost `Flow`s and `Stream`s:

In [30]:
class Costs:
    def __init__(
            self,
            params: dict,
            spans: Spans):
        self.params = params
        self.spans = spans

In [31]:
@rk.update_class(Costs)
class Costs:
    def generate_space_costs(self):
        floor_opex = rk.flux.Flow.from_projection(
            name='Floor-derived Operating Expenses',
            value=self.params['initial_floor_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        facade_opex = rk.flux.Flow.from_projection(
            name='Facade-derived Operating Expenses',
            value=self.params['initial_facade_opex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['opex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.opex = rk.flux.Stream(
            name='Space Operating Expenses',
            flows=[floor_opex, facade_opex],
            period_type=self.params['period_type'])

        floor_capex = rk.flux.Flow.from_projection(
            name='Floor-derived Capital Expenses',
            value=self.params['initial_floor_capex'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['capex_growth_rate']),
            sequence=self.spans.calc_span.to_index(period_type=rk.periodicity.Type.SEMIDECADE)),
            units=currency.units)
        self.floor_capex = floor_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

We will split the generation of space-based costs from the utility-based costs,
as they will be applied to different `Entity` types:

In [32]:
@rk.update_class(Costs)
class Costs:
    def generate_utils_costs(self):
        utils_capex = rk.flux.Flow.from_projection(
            name='Utilities-derived Capital Expenses',
            value=self.params['initial_utility_capex'],
            proj=rk.projection.Extrapolation(
                form=rk.extrapolation.Compounding(
                    rate=self.params['capex_growth_rate']),
                sequence=self.spans.calc_span.to_index(period_type=rk.periodicity.Type.SEMIDECADE)),
            units=currency.units)
        self.utils_capex = utils_capex.trim_to_span(self.spans.calc_span)  # This is to avoid issues with using >yearly periodicity in projection

Now we can run the `Costs` class against 'floor' and 'utilities' `Entity`s:

In [33]:
for floor in floors:
    floor_params = params.copy()

    floor_params['initial_floor_opex'] = floor_opex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['initial_facade_opex'] = facade_opex_per_area_pa[floor['use']] * floor['subtotal_facade_area']
    floor_params['initial_floor_capex'] = floor_capex_per_area_pa[floor['use']] * floor['subtotal_gfa']
    floor_params['opex_growth_rate'] = opex_growth_rate
    floor_params['capex_growth_rate'] = capex_growth_rate
    floor['params'] = floor_params

    floor['events'] = {} if not hasattr(floor, 'events') else floor['events']

    costs = Costs(
        params=floor_params,
        spans=spans)
    costs.generate_space_costs()
    floor['events'].update(dict(
        opex=costs.opex,
        capex=costs.floor_capex))

In [34]:
utilities = scenario.filter_by_type(entity_type='utilities', is_assembly=False)
utilities.get_entities()

{'71fd4551-8e65-4e7a-8e2a-9d18e69146b0': Entity: buildingBcores (Type: utilities),
 '55963d84-702d-423d-be05-c8bb6e88f517': Entity: buildingAcores (Type: utilities),
 '8d9f92cf-5613-4de4-b99b-7681aa76b4cd': Entity: plinthplant (Type: utilities)}

In [35]:
for utility in utilities.get_entities().values():
    utility_params = params.copy()

    utility_params['initial_utility_capex'] = utility_capex_per_vol_pa * utility['volume']
    utility_params['capex_growth_rate'] = capex_growth_rate
    utility['params'] = utility_params

    utility['events'] = {} if not hasattr(utility, 'events') else utility['events']

    costs = Costs(
        params=utility_params,
        spans=spans)
    costs.generate_utils_costs()
    utility['events'].update(dict(
        capex=costs.utils_capex))

In [36]:
list(utilities.get_entities().values())[0]['events']['capex']

date,Utilities-derived Capital Expenses
2005-12-31 00:00:00,"-$1,731,326.78"
2010-12-31 00:00:00,"-$1,783,266.58"


And finally aggregate them according to their relationships type (either
spatial containment or service provision):

In [37]:
# def aggregate_opex(**kwargs):
#     return rk.graph.Entity.aggregate_events(
#         **kwargs,
#         property='opex',
#         period_type=period_type)
# scenario.aggregate(
#     function=aggregate_opex,
#     property='events',
#     label='subtotal_opex',
#     relationship_type='spatiallyContains')

In [38]:
# def aggregate_capex(**kwargs):
#     return rk.graph.Entity.aggregate_events(
#         **kwargs,
#         property='capex',
#         period_type=period_type)
# scenario.aggregate(
#     function=aggregate_capex,
#     property='events',
#     label='subtotal_capex',
#     relationship_type='spatiallyContains')

In [39]:
# def aggregate_utils_capex(**kwargs):
#     return rk.graph.Entity.aggregate_events(
#         **kwargs,
#         property='capex',
#         period_type=period_type)
# scenario.aggregate(
#     function=aggregate_utils_capex,
#     property='events',
#     label='subtotal_utils_capex',
#     relationship_type='services')

If we wish to see the ultimate aggregation of those costs, we can do so by
totalling them at their root `Entity`:

In [40]:
# scenario_opex = scenario.get_roots()['spatiallyContains'][0]['subtotal_opex'].total()
# scenario_capex = scenario.get_roots()['spatiallyContains'][0]['subtotal_capex'].total()
# utils_capex = scenario.get_roots()['services'][0]['subtotal_utils_capex'].total()

In [41]:
# print('Scenario OPEX: ${:,.2f}'.format(scenario_opex))
# print('Scenario CAPEX: ${:,.2f}'.format(scenario_capex))
# print('Utils CAPEX: ${:,.2f}'.format(utils_capex))

To get a sense check if this is reasonable, we can compare the opex and capex
to the PGI. First let's aggregate the PGI for all `Entity`s:

In [42]:
def aggregate_pgis(**kwargs):
    return rk.graph.Entity.aggregate_events(
        **kwargs,
        property='pgi',
        period_type=period_type)

In [43]:
spatial_containment.aggregate(
    property='events',
    function=aggregate_pgis,
    label='subtotal_pgi')

And total it to the root `Entity`:

In [44]:
scenario_pgi = scenario.get_roots()['spatiallyContains'][0]['subtotal_pgi'].total()
print('Scenario PGI: ${:,.2f}'.format(scenario_pgi))

Scenario PGI: $277,563,820.78


The total opex : pgi ratio is:

In [45]:
print('Scenario OpEx as proportion of PGI: {0:.2%}'.format(-scenario_opex / scenario_pgi))

NameError: name 'scenario_opex' is not defined

While the total capex : pgi ratio is:

In [ ]:
print('Scenario CapEx as proportion of PGI: {0:.2%}'.format(-scenario_capex / scenario_pgi))

We can also then chart the relative contributions of each `Entity` to the
sceanrio's Net Annual Cashflows (NACFs):

In [ ]:
# for entity in scenario.get_entities().values():
#     egi_subtotal_value = 0 if not hasattr(entity, 'subtotal_egi') or entity['subtotal_egi'] is None else entity['subtotal_egi'].total()
#     opex_subtotal_value = 0 if not hasattr(entity, 'subtotal_opex') or entity['subtotal_opex'] is None else entity['subtotal_opex'].total()
#     capex_subtotal_value = 0 if not hasattr(entity, 'subtotal_capex') or entity['subtotal_capex'] is None else entity['subtotal_capex'].total()
#
#     entity['subtotal_egi_value'] = egi_subtotal_value / 1000
#     entity['subtotal_opex_value'] = opex_subtotal_value / 1000
#     entity['subtotal_capex_value'] = capex_subtotal_value / 1000
#     entity['subtotal_nacf_value'] = entity['subtotal_egi_value'] + \
#                                     entity['subtotal_opex_value'] + \
#                                     entity['subtotal_capex_value']
#
#
#     #
#     #
#     # flows = list(filter(None, [egi_flow, opex_flow, capex_flow]))
#     # if len(flows) > 0:
#     #     entity['subtotal_nacf'] = rk.flux.Stream(
#     #         name='Net Annual Cashflows',
#     #         flows=flows,
#     #         period_type=period_type)
#     # else:
#     #     entity['subtotal_nacf'] = 0

In [52]:
for entity in scenario.get_entities().values():
    if hasattr(entity, 'events'):
        events = entity['events']
        entity['subtotal_opex'] = events['opex'].total() if hasattr(events, 'opex') else 0
        entity['subtotal_capex'] = events['capex'].total() if hasattr(events, 'capex') else 0
        entity['subtotal_nacf'] = entity['subtotal_egi'].total() if hasattr(entity, 'subtotal_egi') else 0 + entity['subtotal_opex'] + entity['subtotal_capex']


    # egi_subtotal_value = 0 if not hasattr(entity, 'subtotal_egi') or entity['subtotal_egi'] is None else entity['subtotal_egi'].total()
    # opex_subtotal_value = 0 if not hasattr(entity, 'subtotal_opex') or entity['subtotal_opex'] is None else entity['subtotal_opex'].total()
    # capex_subtotal_value = 0 if not hasattr(entity, 'subtotal_capex') or entity['subtotal_capex'] is None else entity['subtotal_capex'].total()
    #
    # entity['subtotal_egi_value'] = egi_subtotal_value / 1000
    # entity['subtotal_opex_value'] = opex_subtotal_value / 1000
    # entity['subtotal_capex_value'] = capex_subtotal_value / 1000
    # entity['subtotal_nacf_value'] = entity['subtotal_egi_value'] + \
    #                                 entity['subtotal_opex_value'] + \
    #                                 entity['subtotal_capex_value']
    #

    #
    #
    # flows = list(filter(None, [egi_flow, opex_flow, capex_flow]))
    # if len(flows) > 0:
    #     entity['subtotal_nacf'] = rk.flux.Stream(
    #         name='Net Annual Cashflows',
    #         flows=flows,
    #         period_type=period_type)
    # else:
    #     entity['subtotal_nacf'] = 0

AttributeError: 'NoneType' object has no attribute 'total'

In [47]:
parking = [e for e in scenario.get_entities().values() if 'parking' in e['name'] and rk.graph.is_entity(e, True)]
parking[1]

Entity: plinthparkingFloor-2 (Type: floor)

In [48]:
parking[1]['events']['egi']

date,Potential Gross Income for cf287254-7d66-4a4b-8a56-f247f51b41db [plinthparkingFloor-2] for cf287254-7d66-4a4b-8a56-f247f51b41db [plinthparkingFloor-2] for cf287254-7d66-4a4b-8a56-f247f51b41db [plinthparkingFloor-2] for cf287254-7d66-4a4b-8a56-f247f51b41db [plinthparkingFloor-2],Vacancy Allowance
2001,$0.00,$0.00
2002,$0.00,$0.00
2003,$0.00,$0.00
2004,$0.00,$0.00
2005,$0.00,$0.00
2006,$0.00,$0.00
2007,$0.00,$0.00
2008,$0.00,$0.00
2009,$0.00,$0.00
2010,$0.00,$0.00


In [51]:
scdf = spatial_containment.to_DataFrame()
scdf = scdf[[
    'name',
    'entityId',
    'subtotal_nacf',
    'subtotal_opex',
    'subtotal_capex',
    ]]
scdf

,name,entityId,subtotal_nacf,subtotal_opex,subtotal_capex
a39ac4c3-094b-4d92-b71d-46c5a857f407,development,a39ac4c3-094b-4d92-b71d-46c5a857f407,NaN,NaN,NaN
981c0743-6a4c-4abd-8a97-2c937e48b897,buildingB,981c0743-6a4c-4abd-8a97-2c937e48b897,NaN,NaN,NaN
e3149444-dfbc-41d0-9eb1-f08785a62da1,plinth,e3149444-dfbc-41d0-9eb1-f08785a62da1,NaN,NaN,NaN
529ce7bc-addf-4bad-8ec5-cbecbd73d4f1,buildingA,529ce7bc-addf-4bad-8ec5-cbecbd73d4f1,NaN,NaN,NaN
29b746f5-42fe-4321-9a4a-8c65c9f0410c,utilities,29b746f5-42fe-4321-9a4a-8c65c9f0410c,NaN,NaN,NaN
3fc29315-81e7-45a9-8241-62938e904d73,buildingBresidential,3fc29315-81e7-45a9-8241-62938e904d73,NaN,NaN,NaN
f508623d-1d81-4082-85ee-5b1e32c7bb10,buildingBparking,f508623d-1d81-4082-85ee-5b1e32c7bb10,NaN,NaN,NaN
6ac8ab6d-a730-44b8-97aa-a5db09b7fbd0,buildingBretail,6ac8ab6d-a730-44b8-97aa-a5db09b7fbd0,NaN,NaN,NaN
71fd4551-8e65-4e7a-8e2a-9d18e69146b0,buildingBcores,71fd4551-8e65-4e7a-8e2a-9d18e69146b0,None,0.0,0.0
c11f68da-0544-487b-9cb9-8a3a3a98a9b8,buildingBresidentialFloor0,c11f68da-0544-487b-9cb9-8a3a3a98a9b8,Effective Gross Income for c11f68da-0544-...,0.0,0.0


In [ ]:
sunburst = spatial_containment.sunburst(property='subtotal_nacf_value')
treemap = spatial_containment.treemap(property='subtotal_nacf_value')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]])
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='aggregate_nacf_chart.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800 * len(fig.data))

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
property = 'subtotal_nacf_value'

df = spatial_containment.to_DataFrame()

df['property_total'] = [item.total() if isinstance(item, rk.flux.Flow) or isinstance(item, rk.flux.Stream)
                        else item for item in df[property]]
df['property_total'] = df['property_total'].abs()
# df['property_total'] = df['property_total'] / 100000

df['trunk_idx'] = df['entityId'].apply(lambda entityId: spatial_containment._get_trunk_index(entityId))
df['color_norm'] = (df['property_total'] - df['property_total'].min()) / (df['property_total'].max() - df['property_total'].min())
df['color'] = df['color_norm'] + (df['trunk_idx'] + 2)

# df['trunk_idx'] = df['trunk_idx'].fillna(0) + 1
df

In [ ]:
from plotly import graph_objects as go
trace = go.Sunburst(
    ids=df['entityId'],
    labels=df['name'],
    parents=df['parent'],
    values=df['property_total'],
    branchvalues='remainder',
    insidetextorientation='radial',
    marker=dict(
        colors=df['color'],
        colorscale='thermal',
        cmin=df['color'].min(),
        cmid=0,
        cmax=df['color'].max()),
    hovertemplate="%{label}<br>" + property + ": %{value:,.2f}<br>",
    sort=False)

In [ ]:
trace

In [ ]:
fig = go.Figure(trace)
py.plot(fig, filename='assets/{0}'.format('foo.html'), auto_open=False)


trace

In [ ]:
floors[1]['subtotal_utils_capex']

In [ ]:
sunburst = spatial_containment.sunburst(property='subtotal_egi')
treemap = spatial_containment.treemap(property='subtotal_egi')
fig = plotly.subplots.make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]])
fig.append_trace(sunburst, row=1, col=1)
fig.append_trace(treemap, row=2, col=1)
filename='test_egi.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800 * len(fig.data))

In [ ]:
sunburst